### **Table of Content:**
1. Transfer Learning
2. Word2Vec (Google)
3. Support Ticket Classification using Word2Vec

#### **Transfer Learning:**

Basically a shortcut in Machine Learning where someone doesn't necessarily need to train a model from scratch, instead we can use simple use someone else's model that has already been trained on a massive dataset for our own problem.

- Useful in NLP as training on huge amount of text takes a lot of time and computing power.
- We can simply use pre-trained models like **Word2Vec** or **GloVe** which already know how words relate to each other and be used to process billions of sentences to create good representations.
- **For example**, if I want to classify text (like identifying whether a support ticket is urgent or not), I need to convert words into numbers. Normally, this would require running algorithms like TF-IDF or training my own embeddings, which can be slow for large datasets. But with pre-trained word vectors, I get ready-made word representations that I can directly plug into my model.
- So, instead of reinventing the wheel, I just "transfer" the learning from these models into my own project. This makes things way faster and often leads to better results since the pre-trained models have already captured complex word relationships from massive datasets.

#### **Word2Vec:**

- Combination of two techniques: 
    - **CBOW (Continuous Bag of Words)**
    - **Skip-gram model**

- Both of these are shallow neural networks that map word(s) to the target variable which is also a word(s).
- Both of these techniques learn weights of the neural network which acts as word vector representations.
- Basically each word is represented as a vector of numbers.

<html><image src="https://thinkingneuron.com/wp-content/uploads/2021/12/image-72.png" alt="Sample word2Vec vectors"></html>

### **CBOW**:
- Continuous Bag of Words
- Predicts the possibility of a word to occur given the words surrounding it, we can consider a single word or a group of words.

### **Skip-gram model**:
- Tries to achieve the reverse of CBOW model
- Tries to predict the source context words (surrounding words) given a target word (the center word)

Note:
- For a larger corpus with higher dimensions, better to use skip-grams but it's a bit slow to train.
- For a smaller corupus and faster training, use CBOW.

<html><image src="https://thinkingneuron.com/wp-content/uploads/2021/12/image-55.png" alt="CBOW vs Skip-Gram"></html>